# EVALUACIÓN DE MÉTODOS DE ENSEMBLE LEARNING (PARTE 1)
**Integrante 1:** Familia de Paralelización (Bagging, Pasting y Random Forests) 

**Tema:** Predicción de Calidad de Aire (Target: PM 2.5)

---

### 1. Introducción y Configuración
En esta sección se implementarán 10 modelos predictivos enfocados en estrategias de reducción de varianza. Según la literatura del Capítulo 7, los métodos de conjunto como **Bagging** y **Random Forests** permiten entrenar predictores de manera independiente y paralela, promediando sus resultados para mejorar la generalización frente a un solo modelo base.

Se utilizará un dataset de calidad de aire (Senamhi) para predecir la concentración de material particulado fino (**PM 2.5**), una variable crítica para la salud pública.

### Configuración y Carga de Datos

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

# Configuración de estilo de gráficos
plt.style.use('seaborn-v0_8-whitegrid')

print("--- 1. CARGA DE DATOS ---")

# Definición de rutas (Estructura: Raíz -> PC1 (csv) | PC3 (notebook))
ruta_local = '../PC1/senamhi_detalle_limpio.csv'
url_raw = 'https://github.com/erickborja26/Analitica-de-Datos/raw/refs/heads/main/PC1/senamhi_detalle_limpio.csv'

try:
    if os.path.exists(ruta_local):
        print(f"[INFO] Cargando desde archivo local: {ruta_local}")
        df = pd.read_csv(ruta_local)
    else:
        print(f"[ALERTA] No se encontró {ruta_local}. Descargando desde GitHub...")
        df = pd.read_csv(url_raw)
    
    print("Carga exitosa.")
    print(f"Dimensiones iniciales: {df.shape}")
    print("Columnas:", df.columns.tolist())
    
except Exception as e:
    print(f"[ERROR] No se pudo cargar la data: {e}")

--- 1. CARGA DE DATOS ---
[INFO] Cargando desde archivo local: ../PC1/senamhi_detalle_limpio.csv
Carga exitosa.
Dimensiones iniciales: (2123, 9)
Columnas: ['Estacion', 'Fecha', 'Hora', 'PM 2.5', 'PM 10', 'SO2', 'NO2', 'O3', 'CO']


### 2. Preprocesamiento (Adaptado al Dataset de SENAMHI)

In [35]:
# --- CONFIGURACIÓN DE DATOS MULTI-SALIDA ---

# 1. Definimos el objetivo integral: Predecir todos los contaminantes
# Esto permite evaluar qué modelo captura mejor la química atmosférica completa.
TARGETS = ['PM 2.5', 'PM 10', 'SO2', 'NO2', 'O3', 'CO']

# 2. Ingeniería de Características (Variables Predictoras)
def procesar_hora(str_hora):
    try:
        return int(str_hora.split(':')[0])
    except:
        return np.nan

# Generamos variables temporales
if 'Hora' in df.columns:
    df['Hora_Num'] = df['Hora'].apply(procesar_hora)

if 'Fecha' in df.columns:
    df['Fecha_Dt'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y', errors='coerce')
    df['Mes'] = df['Fecha_Dt'].dt.month
    df['Dia_Semana'] = df['Fecha_Dt'].dt.dayofweek

# Codificación de Estación (si existe)
if 'Estacion' in df.columns:
    df['Estacion_Code'] = pd.factorize(df['Estacion'])[0]

# Variables de entrada (X)
features = ['Hora_Num', 'Mes', 'Dia_Semana', 'Estacion_Code']

# 3. Limpieza estricta
df_clean = df.dropna(subset=TARGETS + features)

X = df_clean[features]
y = df_clean[TARGETS]

# 4. División de Datos
# Usamos random_state=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("-" * 40)
print(f"Experimental Setup: Multi-Output Regression")
print(f"Inputs (X): {features}")
print(f"Targets (Y): {TARGETS}")
print(f"Muestras de Entrenamiento: {X_train.shape[0]}")
print("-" * 40)

----------------------------------------
Experimental Setup: Multi-Output Regression
Inputs (X): ['Hora_Num', 'Mes', 'Dia_Semana', 'Estacion_Code']
Targets (Y): ['PM 2.5', 'PM 10', 'SO2', 'NO2', 'O3', 'CO']
Muestras de Entrenamiento: 964
----------------------------------------


In [36]:
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor

# Listas globales para acumular los resultados de todos los bloques
metricas_globales = []
print("Listas de métricas inicializadas. Listo para entrenar bloques.")

Listas de métricas inicializadas. Listo para entrenar bloques.


### 3. Implementación: Línea Base y Familia Bagging
A continuación, se entrenan los primeros 4 modelos.
* **Modelo 1 (Línea Base):** Un Árbol de Decisión simple para tener una referencia mínima.
* **Bagging:** Se utiliza `bootstrap=True` (muestreo con reemplazo). Esto permite que algunas instancias se repitan, aumentando ligeramente el sesgo pero reduciendo la varianza.
* **Pasting:** Se utiliza `bootstrap=False` (muestreo sin reemplazo).
* **Bagging Heterogéneo:** Se aplica la técnica de Bagging sobre un estimador diferente (SVR) para demostrar la flexibilidad del algoritmo.

In [37]:
print("--- LÍNEA BASE Y BAGGING ---")

# 1. Decision Tree (Línea Base)
# Usamos un solo árbol para tener un punto de comparación mínimo.
print("Entrenando Modelo 1: Decision Tree...")
m1 = DecisionTreeRegressor(random_state=42)
start = time.time()
m1.fit(X_train, y_train)
t1 = time.time() - start
metricas_globales.append({'Modelo': '1. Decision Tree', 'R2': m1.score(X_test, y_test), 'Tiempo': t1, 'Objeto': m1})

# 2. Bagging Tradicional (Bootstrap=True)
# El muestreo con reemplazo aumenta ligeramente el sesgo pero reduce la varianza.
print("Entrenando Modelo 2: Bagging (Con Reemplazo)...")
m2 = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=50,
    bootstrap=True,  # Bagging
    n_jobs=-1,
    random_state=42
)
start = time.time()
m2.fit(X_train, y_train)
t2 = time.time() - start
metricas_globales.append({'Modelo': '2. Bagging Tree', 'R2': m2.score(X_test, y_test), 'Tiempo': t2, 'Objeto': m2})

# 3. Pasting (Bootstrap=False)
# Muestreo sin reemplazo. Útil en datasets muy grandes.
print("Entrenando Modelo 3: Pasting (Sin Reemplazo)...")
m3 = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=50,
    bootstrap=False, # Pasting
    n_jobs=-1,
    random_state=42
)
start = time.time()
m3.fit(X_train, y_train)
t3 = time.time() - start
metricas_globales.append({'Modelo': '3. Pasting Tree', 'R2': m3.score(X_test, y_test), 'Tiempo': t3, 'Objeto': m3})

# 4. Bagging Heterogéneo (SVR/KNN)
print("Entrenando Modelo 4: Bagging Heterogéneo...")
m4 = BaggingRegressor(
    estimator=KNeighborsRegressor(n_neighbors=5),
    n_estimators=10,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)
start = time.time()
m4.fit(X_train, y_train)
t4 = time.time() - start
metricas_globales.append({'Modelo': '4. Bagging KNN', 'R2': m4.score(X_test, y_test), 'Tiempo': t4, 'Objeto': m4})


--- LÍNEA BASE Y BAGGING ---
Entrenando Modelo 1: Decision Tree...
Entrenando Modelo 2: Bagging (Con Reemplazo)...
Entrenando Modelo 3: Pasting (Sin Reemplazo)...
Entrenando Modelo 4: Bagging Heterogéneo...


### 4. Implementación: Random Forests
El algoritmo **Random Forest** introduce aleatoriedad adicional al buscar la mejor característica dentro de un subconjunto aleatorio, en lugar de todas las disponibles.
Se probarán tres configuraciones:
* **Estándar:** Parámetros por defecto.
* **Profundo:** Aumentando el número de árboles a 300 para capturar patrones más complejos.
* **Regularizado:** Limitando la profundidad del árbol (`max_depth=10`) para controlar el sobreajuste.

In [38]:
print("\n--- RANDOM FORESTS ---")

# 5. Random Forest Estándar
# Configuración por defecto (100 árboles).
print("Entrenando Modelo 5: Random Forest Estándar...")
m5 = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
start = time.time()
m5.fit(X_train, y_train)
t5 = time.time() - start
metricas_globales.append({'Modelo': '5. RF Estándar', 'R2': m5.score(X_test, y_test), 'Tiempo': t5, 'Objeto': m5})

# 6. Random Forest Profundo
# Aumentamos el número de estimadores para capturar patrones más complejos.
print("Entrenando Modelo 6: Random Forest Profundo (300 árboles)...")
m6 = RandomForestRegressor(n_estimators=300, max_depth=None, n_jobs=-1, random_state=42)
start = time.time()
m6.fit(X_train, y_train)
t6 = time.time() - start
metricas_globales.append({'Modelo': '6. RF Profundo', 'R2': m6.score(X_test, y_test), 'Tiempo': t6, 'Objeto': m6})

# 7. Random Forest Regularizado
# Limitamos la profundidad (max_depth) para combatir el overfitting.
print("Entrenando Modelo 7: Random Forest Regularizado...")
m7 = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_leaf=5, n_jobs=-1, random_state=42)
start = time.time()
m7.fit(X_train, y_train)
t7 = time.time() - start
metricas_globales.append({'Modelo': '7. RF Regularizado', 'R2': m7.score(X_test, y_test), 'Tiempo': t7, 'Objeto': m7})



--- RANDOM FORESTS ---
Entrenando Modelo 5: Random Forest Estándar...
Entrenando Modelo 6: Random Forest Profundo (300 árboles)...
Entrenando Modelo 7: Random Forest Regularizado...


### 5. Implementación: Extra-Trees y Selección de Características
Los **Extra-Trees** (Extremely Randomized Trees) utilizan umbrales aleatorios para cada característica, aumentando la velocidad de entrenamiento y reduciendo la varianza.
Además, aprovechamos la capacidad de los bosques para medir la **importancia de las características** y entrenamos un último modelo optimizado.

In [39]:
print("\n--- EXTRA-TREES Y FEATURES ---")

# 8. Extra-Trees Estándar
# Utiliza umbrales aleatorios para cada característica, lo que lo hace más rápido.
print("Entrenando Modelo 8: Extra-Trees Estándar...")
m8 = ExtraTreesRegressor(n_estimators=100, n_jobs=-1, random_state=42)
start = time.time()
m8.fit(X_train, y_train)
t8 = time.time() - start
metricas_globales.append({'Modelo': '8. Extra-Trees', 'R2': m8.score(X_test, y_test), 'Tiempo': t8, 'Objeto': m8})

# 9. Extra-Trees Optimizado
# Ajustamos min_samples_split para suavizar el ruido de los sensores.
print("Entrenando Modelo 9: Extra-Trees Optimizado...")
m9 = ExtraTreesRegressor(n_estimators=200, min_samples_split=10, n_jobs=-1, random_state=42)
start = time.time()
m9.fit(X_train, y_train)
t9 = time.time() - start
metricas_globales.append({'Modelo': '9. ET Optimizado', 'R2': m9.score(X_test, y_test), 'Tiempo': t9, 'Objeto': m9})

# 10. Random Forest con Selección de Características (Log2)
# Forzamos al modelo a ver menos características por división (max_features='log2').
# Esto aumenta la diversidad de los árboles.
print("Entrenando Modelo 10: RF con Selección de Características (Log2)...")
m10 = RandomForestRegressor(n_estimators=100, max_features='log2', n_jobs=-1, random_state=42)
start = time.time()
m10.fit(X_train, y_train)
t10 = time.time() - start
metricas_globales.append({'Modelo': '10. RF Log2 Feat', 'R2': m10.score(X_test, y_test), 'Tiempo': t10, 'Objeto': m10})



--- EXTRA-TREES Y FEATURES ---
Entrenando Modelo 8: Extra-Trees Estándar...
Entrenando Modelo 9: Extra-Trees Optimizado...
Entrenando Modelo 10: RF con Selección de Características (Log2)...


### 5. Visualización de Resultados


In [42]:
import pandas as pd
# Convertimos la lista acumulada en un DataFrame
df_resultados = pd.DataFrame(metricas_globales)
df_resultados = df_resultados.sort_values(by='R2', ascending=False)
df_resultados = df_resultados.drop_duplicates(subset='Modelo', keep='last')
df_resultados = df_resultados.reset_index(drop=True)
df_resultados.index = df_resultados.index + 1

print("\n--- TABLA DE POSICIONES (INTEGRANTE 1) ---")
print(df_resultados[['Modelo', 'R2', 'Tiempo']])


--- TABLA DE POSICIONES (INTEGRANTE 1) ---
                Modelo        R2    Tiempo
1     9. ET Optimizado  0.565693  0.126853
2   7. RF Regularizado  0.519198  0.144613
3       4. Bagging KNN  0.501632  0.072654
4     10. RF Log2 Feat  0.444947  0.104420
5      2. Bagging Tree  0.432354  5.239780
6       6. RF Profundo  0.429598  0.311061
7       5. RF Estándar  0.428532  0.119477
8       8. Extra-Trees  0.223481  0.133094
9      3. Pasting Tree  0.084758  0.184368
10    1. Decision Tree  0.084042  0.007592
